In [182]:
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import pickle
from scipy import stats

# Festlegen globaler Parameter der Simulation

In [183]:
# Breite der Zeitschritte
T_STEP = 15 
# Anzahl zu simulierender Fahrzeuge
EV_TOTAL = 100000
# Anzahl an Tagen die simuliert werden sollen 
D_TOTAL = 1
# Tagtyp: 1 = Werktag, 2 = Samstag, 3 = Sonntag
TYPE_D = 1 
# Ladeszenario: 1 = nur Zuhause, 2 = Zuhause und Arbeit, 3 = Überall
CHARGE_SCEN = 1 
# Ladeleistung 
PCHARGE_SLOW = 3.7 
# durchschnittliche Fahrgeschwindigkeit
# Dummy Wert aus anderem Modell -> anpassen zu späterem Zeitpunkt
EV_SPEED = 19

# Abspeichern der simulierten Fahrzeuge
simulated_evs = []

# Laden der Simulationsdaten

In [184]:
import pickle
import os
root = os.getcwd()+"\\Simulationsdaten"

# Verteilungsfunktion für initiale Abreise
initial_departure_model = pickle.load(open(root+"\\Modell_Initiale_Abfahrtszeit_Werktag.pickle", "rb"))

"""
Übergangswahrscheinlichkeiten:
- Liste von Listen mit einer Liste pro Ausgangszustand mit einem Eintrag pro Zeitschritt (96 Einträge) 
- jeder der 96 Einträge enthält 4 Werte die der jeweiligen relativen Übergangswahrscheinlichkeit entsprechen
    -> sortiert nach numerischer Repräsentation der Zustände
- Beispiel: transition_prob[0] = Liste der Übergangswahrscheinlichkeiten des Zustands Zuhause = 1
            transition_prob[0][10] = [0.2, 0.4, 0.1, 0.3] = [p1_2, p1_3, p1_4, p1_5] im Zeitintervall 9-10(dummy Werte)
"""
transition_prob = pickle.load(open(root+"\\Übergangswahrscheinlichkeiten.pickle", "rb"))


"""
Parameter Verteilungsfunktion Wegstrecken: 
- jeweils m*m Liste wobei Parameter der Verteilungsfunktion der Weglänge von i nach j in Reihe i-1 und Spalte j-1 zu finden sind
- Beispiel: loc[0][1] = Mittelwert der Weglängenverteilung Zustand Zuhause(1) nach Zustand Arbeit(2)
"""
# shape = Formparameter der Verteilung
dist_wd_shape_ij = pickle.load(open(root+"\\Verteilung_Wegstrecken_Werktage_Shape.pickle", "rb"))
# scale = Standardabweichung der Verteilung
dist_wd_scale_ij = pickle.load(open(root+"\\Verteilung_Wegstrecken_Werktage_Scale.pickle", "rb"))
# loc = Mittelwert der Verteilung
dist_wd_loc_ij = pickle.load(open(root+"\\Verteilung_Wegstrecken_Werktage_Loc.pickle", "rb"))

"""
Verteilungsfunktion der Aufenthaltsdauern in den einzelnen Zuständen:
- Liste mit dem jeweiligen Density Estimation Modell des Zustands in sortierter Reihenfolge
- Modell des Zustands i in Index i-1 
- Beispiel: Modell Aufenthaltsdauer Zustand 2(= Arbeit) in stay_duration_model[1]
"""
stay_duration_model = pickle.load(open(root+"\\Modelle_Aufenthaltsdauer_Werktag.pickle", "rb"))



# EV Klasse und Initialisierungslogik

In [185]:
class Electric_Vehicle(object):
    model_df = pd.read_excel(r"C:\Users\thoma\Desktop\ev-modelling-repo\Data\EV_Modelle_Tabelle.xlsx", index_col='Modell')
    segments = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, -9])
    # prob_segment einlesen und nicht bei jeder EV Initialisierung neu definieren 
    
    def __init__(self):
        segment = self.choose_segment()
        model = self.choose_model(segment)
        self.MODEL = model
        
        # Bug da Modelle mehrfach in Liste keine eindeutige Zuordnung sondern Array als output -> fixed -> elegantere Lösung?
        # Segment des Fahrzeugs
        if np.isscalar(Electric_Vehicle.model_df.at[model, "Segment"]):
            self.SEGMENT = Electric_Vehicle.model_df.at[model, "Segment"]
        else:
            self.SEGMENT = Electric_Vehicle.model_df.at[model, "Segment"][0]
        
        # Batteriekapazität in kWh
        if np.isscalar(Electric_Vehicle.model_df.at[model, "Batterie"]):
            self.CAPACITY = Electric_Vehicle.model_df.at[model, "Batterie"]
        else:
            self.CAPACITY = Electric_Vehicle.model_df.at[model, "Batterie"][0]
        
        # Verbrauch in kWh/100km
        if np.isscalar(Electric_Vehicle.model_df.at[model, "Verbrauch"]):
            self.CONSUMPTION = Electric_Vehicle.model_df.at[model, "Verbrauch"]
        else:
            self.CONSUMPTION = Electric_Vehicle.model_df.at[model, "Verbrauch"][0]
        
        # Leistung (notwendig?) 
        if np.isscalar(Electric_Vehicle.model_df.at[model, "Leistung"]):
            self.POWER = Electric_Vehicle.model_df.at[model, "Leistung"]
        else:
            self.POWER = Electric_Vehicle.model_df.at[model, "Leistung"][0]
            
        self.SOC = 100
        self.trip_no = 0
        self.trips = []
    
    def choose_segment(self): 
        # WICHTIG: Später errechnen und als globale Variable in Klasse speichern und nur Zugriff -> Performance
        
        # Beispieldaten der relativen Verteilung der Fharzeugsegmente aus MOP Studie 2016 
        prob_segment = {
        1 : 5.92,
        2 : 19.58,
        3 : 26.21,
        4 : 15.97,
        5 : 2.9,
        6 : 0.58,
        7 : 3.54,
        8 : 1.22,
        9 : 6.37,
        10 : 5.86,
        11 : 3.86,
        12 : 0.58,
        13 : 5.28,
        -9 : 2.13
        }
        
        # Rundungsfehler beseitigen -> sicherstellen dass sich W'keiten zu 1 aufsummieren
        p_ges = sum(val for key, val in prob_segment.items())
        p_rest = 100-p_ges
        prob_segment[-9] = prob_segment[-9]+p_rest
        
        # wähle p zufällig auf Basis gegebener W'keiten
        choice = np.random.choice(Electric_Vehicle.segments, p=[prob_segment[x]/100 for x in Electric_Vehicle.segments])
        
        # Vorerst Wahl des häufigsten Semgents bei keiner Angabe -> später überarbeiten
        if choice == -9: 
            choice = 3
        
        return choice
    
    def choose_model(self, segment):
        models = Electric_Vehicle.model_df
        
        # filtern der infragekommenden Fahrzeuge über Segment
        filt = models["Segment"] == segment
        choices = models[filt]
        
        # Wahl einse zufälligen Fahrzeugs aus der Liste
        pick = np.random.randint(0, len(choices))
        model = choices.index[pick]
        return model
    
    # drive() und charge() testen
    
    def drive(self, distance):
        trip_consumption = distance * (self.CONSUMPTION / 100) 
        self.SOC = self.SOC - (trip_consumption / self.CAPACITY) * 100
        
    def charge(self):       
        # Fahrzeug befindet sich im Zielzustand des letzten Trips
        trip = self.trips[len(self.trips)-1]
        # nötige Zeit zum vollständigen Aufladen in Minuten
        t_charge_full = (((100 - ev.SOC)/100 * ev.CAPACITY) / PCHARGE_SLOW) * 60
        # Ladezeit beschränkt durch Zeit zum vollständigen Aufladen und Länge des Aufenthalts
        t_charge = min(t_charge_full, trip.stay_duration)
        # update SOC über Ladedauer und Ladeleistung
        self.SOC = self.SOC + ((t_charge/60 * PCHARGE_SLOW) / self.CAPACITY) * 100
        trip.charge_start = trip.arrival
        trip.charge_end = round(trip.charge_start + t_charge)

In [196]:
class Trip(object):
    
    def __init__(self, whyfrom, departure):
        self.trip_id = None
        self.trip_no = None
        self.whyfrom = whyfrom
        self.whyto = None
        self.departure = departure
        self.departure_t = round(float(departure/T_STEP))
        self.arrival = None
        self.trip_duration = None
        self.distance = None
        self.stay_duration = None
        self.charge_start = None 
        self.charge_end = None     

In [187]:
for i in range(EV_TOTAL):
    # erzeugen Fahrzeug
    ev = Electric_Vehicle()
    time = 0 
    
    # Fahrten bis Tagesende (24:00)
    while (time < 1440):
        # Unterscheidung zwischen erster Fahrt und restlichen Fahrten
        if len(ev.trips) == 0:
            # Abfahrtszeit = sample der Verteilungsfunktion der initialen Abfahrtszeit
            departure = round(float(initial_departure_model.sample()))
            # erster Trip startet zuhause mit Abfahrtszeit "departure"
            ev_trip = Trip(whyfrom=1, departure=departure)        

        else: 
            # Ursprungszustand des Trips = Zielzustand des letzten Trips
            whyfrom = ev_trip.whyto
            # neuer Trip mit Ausgangszustand = Zielzustand des letzten Trips 
            # Abfahrtszeit = Ankunftszeit des letzten Trips + Aufenthaltsdauer im Zielzustand
            ev_trip = Trip(whyfrom=whyfrom, departure=time)  

        # wähle nächsten Zustand in Abhängigkeit des aktuellen Zustands und des Abfahrtszeitpunkt des Trips
        if ev_trip.whyfrom == 1:
            ev_trip.whyto = np.random.choice([2, 3, 4, 5], p=transition_prob[0][ev_trip.departure_t])
        elif ev_trip.whyfrom == 2:
            ev_trip.whyto = np.random.choice([1, 3, 4, 5], p=transition_prob[1][ev_trip.departure_t])
        elif ev_trip.whyfrom == 3:
            ev_trip.whyto = np.random.choice([1, 2, 4, 5], p=transition_prob[2][ev_trip.departure_t])
        elif ev_trip.whyfrom == 4:
            ev_trip.whyto = np.random.choice([1, 2, 3, 5], p=transition_prob[3][ev_trip.departure_t])
        elif ev_trip.whyfrom == 5:
            ev_trip.whyto = np.random.choice([1, 2, 3, 4], p=transition_prob[4][ev_trip.departure_t])


        # Samplen der Aufenthaltsdauer
        ev_trip.stay_duration = round(float(stay_duration_model[ev_trip.whyto - 1].sample()))

        # Parameter Verteilungsfunktion der Wegstrecke, in Abhängigkeit der Zustandskombination
        # Erläutering Datenstruktur siehe "Parameter der Verteilungsfunktion Wegstrecken:" in "Laden der Simulationsdaten"
        dist_shape = dist_wd_shape_ij[ev_trip.whyfrom - 1][ev_trip.whyto - 1]
        dist_scale = dist_wd_scale_ij[ev_trip.whyfrom - 1][ev_trip.whyto - 1]
        dist_loc = dist_wd_loc_ij[ev_trip.whyfrom - 1][ev_trip.whyto - 1]

        # Samplen der lognorm Verteilungsfunktion der Zustandskombination
        ev_trip.distance = round(stats.lognorm.rvs(s=dist_shape, loc=dist_loc, scale=dist_scale), 1)
        
        # Ladevorgang
        if CHARGE_SCEN == 1:
            if ev.SOC < 100:
                # geladen wird nur wenn sich das Fahrzeug sich Zuhause befindet und sich dort länger als 15 Minuten aufhält
                if ev.trip_no > 0 and ev.trips[len(ev.trips)-1].whyto == 1 and ev_trip.stay_duration > 15:
                    ev.charge()

        # Fahrvorgang -> update SOC 
        ev.drive(ev_trip.distance)
        # Fahrtdauer über Weglänge und durchschnittliche Geschwindigkeit berechnen
        ev_trip.trip_duration = round((ev_trip.distance / EV_SPEED) * 60)
        ev_trip.arrival = ev_trip.departure + ev_trip.trip_duration
        # Trip in Wegetagebuch des Fahrzeugs ablegen
        ev.trips.append(ev_trip)
        # inkrementieren des Wegezählers
        ev.trip_no += 1
        ev_trip.trip_no = ev.trip_no
        # Trip_id = Fahrzeugnummer(= i).Tripnummer 
        ev_trip.trip_id = float(f"{i}.{ev.trip_no}")
        # Update der aktuellen Zeit
        time = round(ev_trip.arrival + ev_trip.stay_duration)
    
    # Ladevorgang bei Ankunft vom letzten Trip
    if CHARGE_SCEN == 1:
        if ev.SOC < 100:
            # geladen wird nur wenn sich das Fahrzeug sich Zuhause befindet und sich dort länger als 15 Minuten aufhält
            if ev.trip_no > 0 and ev.trips[len(ev.trips)-1].whyto == 1 and ev_trip.stay_duration > 15:
                ev.charge()
    
    # Speichern des Fahrzeugs
    simulated_evs.append(ev)
    
    

In [190]:
from collections import defaultdict

# Dictionary Key : List 
total_trips_dict = defaultdict(list)

# speichern der Trips jedes einzelnen Fahrzeugs im dict
for ev in simulated_evs:
    for trip in ev.trips:
        # .__dict__.items() returnt Dictionary mit allen Member Variablen und dazugehörigen Werten des Objekts
        for key, val in trip.__dict__.items():
            total_trips_dict[key].append(val)

# umwandeln in DataFrame
data = pd.DataFrame(total_trips_dict)

In [192]:
data.head(20)

,trip_id,trip_no,whyfrom,whyto,departure,departure_t,trip_duration,arrival,distance,stay_duration,charge_start,charge_end
0,0.1,1,1,2,313.0,21,18.0,331.0,5.8,447,NaN,NaN
1,0.2,2,2,1,778.0,52,35.0,813.0,11.1,1355,813.0,848.0
2,1.1,1,1,2,509.0,34,128.0,637.0,40.6,571,NaN,NaN
3,1.2,2,2,1,1208.0,81,309.0,1517.0,97.7,728,1517.0,2078.0
4,2.1,1,1,4,427.0,28,86.0,513.0,27.2,628,NaN,NaN
5,2.2,2,4,1,1141.0,76,16.0,1157.0,5.0,974,1157.0,1231.0
6,3.1,1,1,2,421.0,28,18.0,439.0,5.8,574,NaN,NaN
7,3.2,2,2,1,1013.0,68,37.0,1050.0,11.6,-2,NaN,NaN
8,3.3,3,1,3,1048.0,70,92.0,1140.0,29.0,0,NaN,NaN
9,3.4,4,3,1,1140.0,76,8.0,1148.0,2.6,827,1148.0,1296.0
